##### DART Report: PDS Organizations Location Discrepancies


* Business Owner: Ana Riquezes
* Script Owner: Anna Soloveva
* Jira Ticket: [CQA-1212](https://h1insights.atlassian.net/browse/CQA-1212)

In [0]:
%pip install --upgrade pip
%pip install pyodbc
%pip install unidecode

Python interpreter will be restarted.
Requirement already satisfied: pip in /local_disk0/.ephemeral_nfs/envs/pythonEnv-4e9280ea-646c-4cae-a2fa-6df894718699/lib/python3.8/site-packages (22.3.1)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: pyodbc in /databricks/python3/lib/python3.8/site-packages (4.0.35)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: unidecode in /databricks/python3/lib/python3.8/site-packages (1.3.6)
Python interpreter will be restarted.

### Setup

In [0]:
dbutils.widgets.dropdown("gdriveUpload", "False", ["True", "False"])
dbutils.widgets.dropdown("jiraPost", "False", ["True", "False"])

gdriveUpload = dbutils.widgets.get("gdriveUpload")
jiraPost = dbutils.widgets.get("jiraPost")

gdriveUpload = eval(gdriveUpload)
jiraPost = eval(jiraPost)

In [0]:
import sys
sys.path.insert(0, '/dbfs/FileStore/shared_uploads/anna.soloveva@h1.co/')
from sql_library2 import Tesseract

In [0]:
import pandas as pd 
import numpy as np
from collections import Counter
import re
import json

from unidecode import unidecode

import sys
sys.path.insert(0, '/Workspace/Repos/pablo.flores@h1.co/sql_tesseract/')
sys.path.insert(0,'/Workspace/Repos/pablo.flores@h1.co/databricks_helpers/')
sys.path.insert(0, '/dbfs/FileStore/shared_uploads/pablo.flores@h1.co/models')
from gdrive_uploader import UploadGDrive
from jira_poster import JiraIssue, JiraSecrets

from databricks_saver import Saver
from summary_creator import Summary
from sql_library import Tesseract

import jira as js
from datetime import date
import datetime as dt
import pygsheets
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account
as_of = date.today()

### Pull Data

In [0]:
# pds_loc_disc = f"""
# SELECT ppm.PeopleID ,ppm.FullName , ppm.AffiliationsCount ,pipm.[source] ,pipm.InstitutionName , pipm.MasterOrganizationID , 
# ol.ParentId AS h1_organization_id, o.OrganizationId ,hp.ShoreOrganizationID ,hp.HospitalID, hp.CreatedDate ,hp.HospitalName ,u.OrganizationName as OrganizationName_based_on_ShoreOrganizationID, hp.HospitalAddress1, hp.ShoreCityName ,hp.ShoreStateName ,hp.ShoreCountryName ,hp.zipcode ,
# ol.OrganizationName as LinkOrganizationName, ol.OrganizationCode, ol.SourceId , ol.OtherCodeTypeId , 
# o.OrganizationName as MasteredOrganizationName, a.Address1 , a.Address2 ,a.CityName ,a.State , a.Country, a.ZipCode --C.HospitalIDCount, 
# FROM Tesseract.curie.People_PreMaster ppm 
# LEFT JOIN Tesseract.curie.PeopleInstitution_PreMaster pipm WITH (NOLOCK) ON ppm.PeopleID = pipm.PeopleID 
# INNER JOIN pd.Hospital_Premaster hp WITH(NOLOCK) ON pipm.MasterOrganizationID = hp.ShoreOrganizationID
# INNER JOIN Tesseract.om.OrganizationLink ol WITH(NOLOCK) ON ol.OrganizationCode = hp.HospitalID AND sourceid = 14 AND OtherCodeTypeId = 19
# LEFT JOIN om.Organization o WITH(NOLOCK) ON o.OrganizationId = ol.ParentId AND o.IsDeleted = 0 AND o.IsActive  = 1
# LEFT JOIN om.OrganizationAddress a WITH(NOLOCK) ON o.OrganizationId=a.ParentId AND a.IsDeleted = 0 AND a.IsActive  = 1
# --JOIN (SELECT ol.ParentId AS h1_organization_id, count( distinct hp.HospitalID) as HospitalIDCount
# --FROM pd.Hospital_Premaster hp WITH(NOLOCK) 
# --INNER JOIN Tesseract.om.OrganizationLink ol WITH(NOLOCK) ON OrganizationCode = hp.HospitalID AND sourceid = 14 AND OtherCodeTypeId = 19
# --GROUP BY ol.ParentId)C
# --HAVING count( hp.HospitalID) > 1)C
# --ON C.h1_organization_id = ol.ParentId 
# INNER JOIN ( 
# SELECT o.OrganizationId, o.OrganizationName, a.State
# FROM  om.Organization o
# LEFT JOIN om.OrganizationAddress a WITH(NOLOCK) ON o.OrganizationId=a.ParentId AND a.IsDeleted = 0 AND a.IsActive  = 1)u
# ON hp.ShoreOrganizationID = u.OrganizationId
# WHERE hp.IsActive = 1 AND hp.IsDeleted  = 0 AND  pipm.[source] in (13,14) AND ol.ParentId in (SELECT ol.ParentId
# FROM pd.Hospital_Premaster hp WITH(NOLOCK) 
# INNER JOIN Tesseract.om.OrganizationLink ol WITH(NOLOCK) ON OrganizationCode = hp.HospitalID AND sourceid = 14 AND OtherCodeTypeId = 19
# INNER JOIN om.Organization o WITH(NOLOCK) ON o.OrganizationId = ol.ParentId AND o.IsDeleted = 0 AND o.IsActive  = 1
# INNER JOIN om.OrganizationAddress a WITH(NOLOCK) ON o.OrganizationId=a.ParentId AND a.IsDeleted = 0 AND a.IsActive  = 1
# WHERE  hp.ShoreOrganizationID<>ol.ParentId)  -- hp.ShoreOrganizationID<>ol.ParentId  hp.HospitalAddress1<>a.Address1 AND hp.zipcode <> a.ZipCode
# --AND ol.ParentId = 53 
# -- AND hp.CreatedDate > '2022-01-01'
# ORDER BY hp.HospitalName  ---ol.ParentId --
# """

# pds_loc_disc = Tesseract.consult(pds_loc_disc)
# display(pds_loc_disc)

In [0]:
# PDS ZipCode is not null but not matching to Curie

pds_loc_disc = f"""
SELECT distinct '' as Date, '' as Assignee, pipm.MasterOrganizationID , pipm.InstitutionName, apm.AddressLine1, apm.State,apm.Zip, hp.HospitalID as PDS_HospitalID, hp.HospitalName as PDS_HospitalName, hp.HospitalAddress1 as PDS_HospitalAddress1,  hp.ShoreOrganizationID, hp.ShoreCityName, hp.ShoreStateName, hp.ShoreCountryName  ,  hp.zipcode as PDS_zipcode,
'' as Validated_PDS_HospitalName, '' as Validated_PDS_HospitalAddress1,	'' as Validated_ShoreCityName,'' as Validated_ShoreStateName, '' as Validated_ShoreCountryName,'' as Validated_PDS_zipcode, '' as Comments
FROM Tesseract.curie.People_PreMaster ppm 
LEFT JOIN Tesseract.curie.PeopleInstitution_PreMaster pipm WITH (NOLOCK) ON ppm.PeopleID = pipm.PeopleID  AND pipm.IsActive = 1 AND pipm.IsDeleted  = 0
LEFT JOIN Tesseract.curie.PeopleInstitutionAffiliation_PreMaster pia WITH (NOLOCK) ON pipm.PeopleInstitutionID = pia.ParentID
LEFT JOIN Tesseract.curie.InstitutionAddress_PreMaster iapm WITH (NOLOCK) ON iapm.InstitutionID = pipm.InstitutionID  AND iapm.IsActive = 1
LEFT JOIN Tesseract.curie.Address_PreMaster apm WITH (NOLOCK) ON apm.AddressID = iapm.AddressID 
LEFT JOIN Tesseract.curie.PeopleInstitutionAffiliation_PreMaster piapm WITH (NOLOCK) ON pipm.PeopleInstitutionID = piapm.ParentID 
INNER JOIN pd.Hospital_Premaster hp WITH(NOLOCK) ON pipm.MasterOrganizationID = hp.ShoreOrganizationID AND hp.IsActive = 1 AND hp.IsDeleted  = 0
WHERE pipm.[source] in (13,14) AND ppm.IsActive = 1 AND ppm.IsDeleted  = 0
AND apm.Zip <> hp.zipcode 
--AND apm.State <> hp.ShoreStateName
AND hp.HospitalAddress1 <> apm.AddressLine1
ORDER BY pipm.InstitutionName --ppm.PeopleID --

"""

pds_loc_disc = Tesseract.consult(pds_loc_disc)
display(pds_loc_disc)

Result Shape: (83, 22)

Date,Assignee,MasterOrganizationID,InstitutionName,AddressLine1,State,Zip,PDS_HospitalID,PDS_HospitalName,PDS_HospitalAddress1,ShoreOrganizationID,ShoreCityName,ShoreStateName,ShoreCountryName,PDS_zipcode,Validated_PDS_HospitalName,Validated_PDS_HospitalAddress1,Validated_ShoreCityName,Validated_ShoreStateName,Validated_ShoreCountryName,Validated_PDS_zipcode,Comments
,,338154,Ascension Columbia St. Mary's Hospital Milwaukee,2301 North Lake Dr,Wisconsin,53211,100,Columbia St Marys Health System (Ascension Wisconsin),101 South Hanley Road,338154,null,null,null,63105,,,,,,,
,,338154,Ascension Columbia St. Mary's Hospital Milwaukee,2301 North Lake Dr,Wisconsin,53211,186958,Columbia St Marys Health System (Ascension Wisconsin),101 South Hanley Road,338154,null,null,null,63105,,,,,,,
,,5543,Ascension St. John Medical Center,7501 Riverside Pkwy,Oklahoma,74136,1666,St John Health System,St. John Health System Corporate Offices,5543,null,null,null,74104,,,,,,,
,,5543,Ascension St. John Medical Center,7501 Riverside Pkwy,Oklahoma,74136,188523,St John Health System,St. John Health System Corporate Offices,5543,null,null,null,74104,,,,,,,
,,5073,Avera Health,3900 West Avera Drive,South Dakota,57108,188723,Avera Health,1325 S Cliff Avenue.,5073,null,null,null,57117,,,,,,,
,,5073,Avera Health,3900 West Avera Drive,South Dakota,57108,2212,Avera Health,1325 S Cliff Avenue.,5073,null,null,null,57117,,,,,,,
,,436,Bon Secours Richmond Community Hospital,1500 N. 28th Street,Virginia,23223,188173,Bon Secours Richmond Health System,2000 West Baltimore Street,436,null,null,null,21223,,,,,,,
,,436,Bon Secours Richmond Community Hospital,1500 N. 28th Street,Virginia,23223,4208,Bon Secours Richmond Health System,2000 West Baltimore Street,436,null,null,null,21223,,,,,,,
,,691,CHI Health Good Samaritan,10 E 31st St,Nebraska,68847,1771,CHI Health Good Samaritan (FKA Good Samaritan Hospital),"P O BOX 1990, 10 EAST 31ST ST",691,null,null,null,68848,,,,,,,
,,691,CHI Health Good Samaritan,10 E 31st St,Nebraska,68847,186797,CHI Health Good Samaritan (FKA Good Samaritan Hospital),"P O BOX 1990, 10 EAST 31ST ST",691,null,null,null,68848,,,,,,,


In [0]:
# PDS ZipCode is null

pds_loc_disc1 = f"""
SELECT distinct '' as Date, '' as Assignee, ol.ParentId, ol.OrganizationCode, pipm.[source], pipm.MasterOrganizationID , pipm.InstitutionName, apm.AddressLine1, apm.State,apm.Zip, hp.HospitalID as PDS_HospitalID, hp.HospitalName as PDS_HospitalName, hp.HospitalAddress1 as PDS_HospitalAddress1, hp.ShoreOrganizationID, hp.ShoreCityName, hp.ShoreStateName, hp.ShoreCountryName  ,  hp.zipcode as PDS_zipcode,'' as Validated_PDS_HospitalName, '' as Validated_PDS_HospitalAddress1,	'' as Validated_ShoreCityName,'' as Validated_ShoreStateName, '' as Validated_ShoreCountryName,'' as Validated_PDS_zipcode, '' as Comments
FROM pd.Hospital_Premaster hp WITH(NOLOCK) --ON pipm.MasterOrganizationID = hp.ShoreOrganizationID AND hp.IsActive = 1 AND hp.IsDeleted  = 0
INNER JOIN om.OrganizationLink ol WITH(NOLOCK) ON ol.OrganizationCode = hp.HospitalID AND sourceid = 14 AND OtherCodeTypeId = 19
LEFT JOIN Tesseract.curie.PeopleInstitution_PreMaster pipm WITH (NOLOCK) ON ol.ParentId = pipm.MasterOrganizationID AND pipm.IsActive = 1 AND pipm.IsDeleted  = 0
LEFT JOIN Tesseract.curie.PeopleInstitutionAffiliation_PreMaster pia WITH (NOLOCK) ON pipm.PeopleInstitutionID = pia.ParentID AND pipm.IsActive = 1 AND pipm.IsDeleted  = 0
LEFT JOIN Tesseract.curie.InstitutionAddress_PreMaster iapm WITH (NOLOCK) ON iapm.InstitutionID = pipm.InstitutionID  AND iapm.IsActive = 1
LEFT JOIN Tesseract.curie.Address_PreMaster apm WITH (NOLOCK) ON apm.AddressID = iapm.AddressID 
LEFT JOIN Tesseract.curie.PeopleInstitutionAffiliation_PreMaster piapm WITH (NOLOCK) ON pipm.PeopleInstitutionID = piapm.ParentID
WHERE pipm.[source] in (13,14)
AND hp.zipcode is null
--AND apm.Zip <> hp.zipcode 
--AND apm.State <> hp.ShoreStateName
AND hp.HospitalAddress1 <> apm.AddressLine1
ORDER BY pipm.InstitutionName --ppm.PeopleID --

"""

pds_loc_disc1 = Tesseract.consult(pds_loc_disc1)
display(pds_loc_disc1)

Result Shape: (41, 25)

Date,Assignee,ParentId,OrganizationCode,source,MasterOrganizationID,InstitutionName,AddressLine1,State,Zip,PDS_HospitalID,PDS_HospitalName,PDS_HospitalAddress1,ShoreOrganizationID,ShoreCityName,ShoreStateName,ShoreCountryName,PDS_zipcode,Validated_PDS_HospitalName,Validated_PDS_HospitalAddress1,Validated_ShoreCityName,Validated_ShoreStateName,Validated_ShoreCountryName,Validated_PDS_zipcode,Comments
,,17598,148881,14,17598,AMITA Health Alexian Brothers Behavioral Health Hospital,1650 Moon Lake Blvd.,Illinois,60169,148881,AMITA Health Alexian Brothers Behavioral Health Hospital,"1650 Moon Lake Blvd., Hoffman Estates, Illinois, 60169",17598,null,null,null,null,,,,,,,
,,338154,148915,14,338154,Ascension Columbia St. Mary's Hospital Milwaukee,2301 North Lake Dr,Wisconsin,53211,148915,Ascension Columbia St Marys Hospital - Ozaukee,"13111 N Port Washington Rd, Mequon, WI 53097",338154,null,null,null,null,,,,,,,
,,338154,148977,14,338154,Ascension Columbia St. Mary's Hospital Milwaukee,2301 North Lake Dr,Wisconsin,53211,148977,Ascension Columbia St Marys Hospital - Milwaukee,"2301 N Lake Dr, Milwaukee, WI 53211, United States",338154,null,null,null,null,,,,,,,
,,5703,148884,14,5703,Ascension SE Wisconsin Hospital - Franklin Campus,10101 South 27th Street,Wisconsin,53132,148884,Ascension Wheaton Franciscan Healthcare - Franklin,"10101 S 27th St, Franklin, WI 53132",5703,null,null,null,null,,,,,,,
,,11501,148929,14,11501,Aurora Psychiatric Hospital,1220 Dewey Ave,Wisconsin,53213,148929,Aurora Psychiatric Hospital,"1220 Dewey Ave, Wauwatosa, Wisconsin, 53213",11501,null,null,null,null,,,,,,,
,,3739,148888,14,3739,Baylor Scott & White Medical Center - Brenham,700 Medical Parkway,Texas,77833,148888,Baylor Scott & White Medical Center - Brenham,"700 MEDICAL PARKWAY, Brenham, Texas, 77833",3739,null,null,null,null,,,,,,,
,,364,148943,14,364,Bell Hospital,901 Lakeshore Drive,Michigan,49849,148943,UP Health System - Bell,"901 LAKESHORE DRIVE, Ishpeming, Michigan, 49849",364,null,null,null,null,,,,,,,
,,17605,149000,14,17605,CenterPointe Hospital,4801 Weldon Spring Parkway,Missouri,63304,149000,CenterPointe Hospital,"4801 Weldon Spring Parkway, Saint Charles, Missouri, 63304",17605,null,null,null,null,,,,,,,
,,11497,148994,14,11497,Chicago Lakeshore Hospital,4840 N Marine Dr.,Illinois,60640,148994,Aurora Chicago Lakeshore Hospital,"4840 N Marine Dr., Chicago, Illinois, 60640",11497,null,null,null,null,,,,,,,
,,5170,148950,14,5170,Christus Southeast Texas - St. Elizabeth,2830 Calder Street,Texas,77702,148950,CHRISTUS Southeast Texas,"2830 Calder Street, Beaumont, Texas, 77702",5170,null,null,null,null,,,,,,,


### Create Gsheet and Jira Ticket

In [0]:
if gdriveUpload == True:
  sheetstructure = { '1.PDSOrgs (with ZipCode)':pds_loc_disc,
                     '2.PDS Orgs (No ZipCode)':pds_loc_disc1,}
  
  upload = UploadGDrive(client_secrets = '/dbfs/FileStore/tables/api_integration_350021_c857e9d75586.json',  
                        folder_id = '1oaPpR44h45GrTYCXpKuYoFLdialPd7xV', 
                        file_name = f'PDS Organizations Location Discrepancies' , 
                        sheetstructure = sheetstructure )
  upload.run()
  message = upload.get_message()
  print(message)

In [0]:
if jiraPost == True:
  notebook_link = "https://dbc-593c2c2d-f6be.cloud.databricks.com/?o=960084232686341#notebook/1814144139191266/command/3555372334039765"
  job_link = "https://dbc-593c2c2d-f6be.cloud.databricks.com/?o=960084232686341#job/146848697741841/runs"
  
  details = {'project': 'DART', 
           'parent':'DART-54', 
           'issuetype':'Sub-task' ,
           'title':f'PDS Organizations Location Discrepancies', 
           'description':f"""Issue: Find current work affiliation for HCPs with only student affiliation who have already graduated.,
                           
                           {message},
                           
                           Databricks Notebook: {notebook_link},
                           Databricks Job: {job_link}""", 
           'assignee': '557058:b4d652ba-8f41-491a-b181-f9a1b938ec5e'} # Assignee: Venu Bolla
  
  credentials = JiraSecrets(user_email = 'ana.riquezes@h1.co', api_token = '0LbBY6F49qwVVHfCyAG800B4')
  JiraIssue(credentials = credentials).post(details = details)
  print(JiraIssue)